In [212]:
import pandas as pd
import requests as req
from datetime import datetime, timezone
from pulp import *

In [213]:
uri = 'https://fantasy.premierleague.com/api/bootstrap-static/'
r = req.get(uri)
json = r.json()
json.keys()

dict_keys(['events', 'game_settings', 'phases', 'teams', 'total_players', 'elements', 'element_stats', 'element_types'])

In [214]:
# Get data of note from the API
elements_df = pd.DataFrame(json['elements'])
elements_df.head()

,chance_of_playing_next_round,chance_of_playing_this_round,code,cost_change_event,cost_change_event_fall,cost_change_start,cost_change_start_fall,dreamteam_count,element_type,ep_next,...,threat_rank,threat_rank_type,ict_index_rank,ict_index_rank_type,corners_and_indirect_freekicks_order,corners_and_indirect_freekicks_text,direct_freekicks_order,direct_freekicks_text,penalties_order,penalties_text
0,NaN,None,58822,0,0,0,0,0,2,2.3,...,240,81,199,63,2.0,,3.0,,NaN,
1,NaN,None,84450,0,0,0,0,0,3,2.0,...,148,85,102,63,NaN,,NaN,,NaN,
2,NaN,None,153256,0,0,0,0,0,3,1.5,...,292,145,287,130,NaN,,NaN,,NaN,
3,NaN,None,156074,0,0,0,0,0,2,2.3,...,293,113,296,108,NaN,,NaN,,NaN,
4,NaN,None,167199,0,0,0,0,0,3,2.0,...,100,60,132,76,NaN,,4.0,,NaN,


In [215]:
element_types_df = pd.DataFrame(json['element_types'])
element_types_df.head()

,id,plural_name,plural_name_short,singular_name,singular_name_short,squad_select,squad_min_play,squad_max_play,ui_shirt_specific,sub_positions_locked,element_count
0,1,Goalkeepers,GKP,Goalkeeper,GKP,2,1,1,True,[12],55
1,2,Defenders,DEF,Defender,DEF,5,3,5,False,[],206
2,3,Midfielders,MID,Midfielder,MID,5,2,5,False,[],237
3,4,Forwards,FWD,Forward,FWD,3,1,3,False,[],60


In [216]:
teams_df = pd.DataFrame(json['teams'])
teams_df.head()

,code,draw,form,id,loss,name,played,points,position,short_name,...,team_division,unavailable,win,strength_overall_home,strength_overall_away,strength_attack_home,strength_attack_away,strength_defence_home,strength_defence_away,pulse_id
0,3,0,None,1,0,Arsenal,0,0,0,ARS,...,None,False,0,1200,1270,1150,1210,1190,1220,1
1,7,0,None,2,0,Aston Villa,0,0,0,AVL,...,None,False,0,1090,1100,1140,1110,1090,1090,2
2,91,0,None,3,0,Bournemouth,0,0,0,BOU,...,None,False,0,1050,1050,1080,1130,1060,1100,127
3,94,0,None,4,0,Brentford,0,0,0,BRE,...,None,False,0,1090,1100,1120,1150,1080,1120,130
4,36,0,None,5,0,Brighton,0,0,0,BHA,...,None,False,0,1100,1090,1160,1160,1100,1120,131


In [217]:
elements_df['position'] = elements_df.element_type.map(element_types_df.set_index('id').singular_name_short)
elements_df['team'] = elements_df.team.map(teams_df.set_index('id').name)
elements_df['name'] = elements_df['first_name'] + ' ' + elements_df['second_name']

elements_df['value'] = elements_df.value_season.astype(float)

In [218]:
players_df = elements_df[
    [
        'name',
        'team',
        'goals_scored',
        'assists',
        'total_points',
        'minutes',
        'goals_conceded',
        'creativity',
        'influence',
        'threat',
        'bonus',
        'bps',
        'ict_index',
        'clean_sheets',
        'red_cards',
        'yellow_cards',
        'selected_by_percent',
        'value',
        'position'
    ]
]

players_df

,name,team,goals_scored,assists,total_points,minutes,goals_conceded,creativity,influence,threat,bonus,bps,ict_index,clean_sheets,red_cards,yellow_cards,selected_by_percent,value,position
0,Cédric Alves Soares,Arsenal,1,1,48,1481,27,327.1,318.4,111.0,3,292,75.8,3,0,3,0.3,10.7,DEF
1,Granit Xhaka,Arsenal,1,2,60,2327,38,522.9,395.4,262.0,3,334,118.2,6,1,10,0.5,12.0,MID
2,Mohamed Elneny,Arsenal,0,2,27,801,17,131.5,163.2,49.0,0,156,34.4,0,0,1,1.2,6.0,MID
3,Rob Holding,Arsenal,1,0,29,840,16,8.6,253.0,49.0,2,152,31.1,2,1,2,0.2,6.4,DEF
4,Thomas Partey,Arsenal,2,2,71,2027,25,275.5,359.6,394.0,4,345,103.0,10,0,6,0.5,14.2,MID
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
553,Yerson Mosquera Valdelamar,Wolves,0,0,0,0,0,0.0,0.0,0.0,0,0,0.0,0,0,0,1.2,0.0,DEF
554,João Filipe Iria Santos Moutinho,Wolves,2,1,96,2970,40,660.9,527.8,144.0,8,511,133.2,11,0,4,2.1,19.2,MID
555,Nathan Collins,Wolves,2,0,55,1670,26,78.2,436.0,254.0,1,272,77.0,5,1,2,0.3,12.2,DEF
556,Jackson Smith,Wolves,0,0,0,0,0,0.0,0.0,0.0,0,0,0.0,0,0,0,0.1,0.0,GKP


In [219]:
players_df.loc[players_df['total_points'] == 0]

,name,team,goals_scored,assists,total_points,minutes,goals_conceded,creativity,influence,threat,bonus,bps,ict_index,clean_sheets,red_cards,yellow_cards,selected_by_percent,value,position
20,Lucas Torreira di Pascua,Arsenal,0,0,0,0,0,0.0,0.0,0.0,0,0,0.0,0,0,0,0.5,0.0,MID
22,Matt Turner,Arsenal,0,0,0,0,0,0.0,0.0,0.0,0,0,0.0,0,0,0,0.8,0.0,GKP
23,Fábio Ferreira Vieira,Arsenal,0,0,0,0,0,0.0,0.0,0.0,0,0,0.0,0,0,0,0.4,0.0,MID
24,William Saliba,Arsenal,0,0,0,0,0,0.0,0.0,0.0,0,0,0.0,0,0,0,1.8,0.0,DEF
25,Marcus Oliveira Alencar,Arsenal,0,0,0,0,0,0.0,0.0,0.0,0,0,0.0,0,0,0,0.1,0.0,MID
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
532,Conor Coventry,West Ham,0,0,0,0,0,0.0,0.0,0.0,0,0,0.0,0,0,0,0.0,0.0,MID
551,Matija Šarkić,Wolves,0,0,0,0,0,0.0,0.0,0.0,0,0,0.0,0,0,0,2.9,0.0,GKP
553,Yerson Mosquera Valdelamar,Wolves,0,0,0,0,0,0.0,0.0,0.0,0,0,0.0,0,0,0,1.2,0.0,DEF
556,Jackson Smith,Wolves,0,0,0,0,0,0.0,0.0,0.0,0,0,0.0,0,0,0,0.1,0.0,GKP


In [220]:
# Sort by value
players_df.sort_values('value', ascending=False).head(10)

,name,team,goals_scored,assists,total_points,minutes,goals_conceded,creativity,influence,threat,bonus,bps,ict_index,clean_sheets,red_cards,yellow_cards,selected_by_percent,value,position
314,Alisson Ramses Becker,Liverpool,0,1,176,3240,24,10.0,721.4,0.0,15,782,73.1,20,0,0,19.7,32.0,GKP
42,Matty Cash,Aston Villa,4,3,147,3377,50,432.1,754.8,534.0,11,623,172.1,13,0,8,20.5,29.4,DEF
14,Gabriel dos Santos Magalhães,Arsenal,5,0,146,3063,38,131.6,746.8,470.0,16,682,134.7,13,1,6,8.6,29.2,DEF
537,José Malheiro de Sá,Wolves,0,1,146,3285,40,30.0,947.0,0.0,11,722,97.5,11,0,3,4.6,29.2,GKP
477,Hugo Lloris,Spurs,0,0,158,3420,40,23.0,746.2,0.0,11,703,76.9,16,0,2,4.9,28.7,GKP
342,João Cancelo,Man City,1,11,201,3227,25,786.9,796.0,851.0,27,829,243.6,19,0,7,45.1,28.7,DEF
309,Joel Matip,Liverpool,3,3,170,2790,21,195.3,711.4,509.0,20,763,141.7,17,0,2,2.7,28.3,DEF
313,Virgil van Dijk,Liverpool,3,3,183,3060,21,150.3,726.4,455.0,13,803,133.1,21,0,3,13.6,28.2,DEF
343,Ederson Santana de Moraes,Man City,0,0,155,3330,26,0.0,561.2,0.0,4,701,56.1,20,0,3,16.2,28.2,GKP
124,Robert Sánchez,Brighton,0,0,126,3330,42,30.0,811.0,0.0,8,679,84.0,11,1,3,13.0,28.0,GKP


In [221]:
data = players_df[
    [
        'name',
        'team',
        'position',
        'total_points',
        'value',
        'goals_scored',
        'assists',
        'minutes',
        'goals_conceded',
        'creativity',
        'influence',
        'threat',
        'bonus',
        'bps',
        'ict_index',
        'clean_sheets',
        'red_cards',
        'yellow_cards',
        'selected_by_percent'
    ]
]
data

,name,team,position,total_points,value
0,Cédric Alves Soares,Arsenal,DEF,48,10.7
1,Granit Xhaka,Arsenal,MID,60,12.0
2,Mohamed Elneny,Arsenal,MID,27,6.0
3,Rob Holding,Arsenal,DEF,29,6.4
4,Thomas Partey,Arsenal,MID,71,14.2
...,...,...,...,...,...
553,Yerson Mosquera Valdelamar,Wolves,DEF,0,0.0
554,João Filipe Iria Santos Moutinho,Wolves,MID,96,19.2
555,Nathan Collins,Wolves,DEF,55,12.2
556,Jackson Smith,Wolves,GKP,0,0.0


In [222]:
POS = data.position.unique()
CLUBS = data.team.unique()
BUDGET = 1000
pos_available = {
    'DEF':5,
    'FWD':3,
    'MID':5,
    'GKP':2
}

names = [data.name[i] for i in data.index]
teams = [data.team[i] for i in data.index]
positions = [data.position[i] for i in data.index]
prices = [data.value[i] for i in data.index]
points = [data.total_points[i] for i in data.index]
players = [LpVariable("player_" + str(i), cat="Binary") for i in data.index]

In [223]:
prob = LpProblem("FPL_Player_Choices", LpMaximize)

In [224]:
# Objective function
prob += lpSum(players[i] * points[i] for i in range(len(data)))

In [225]:
# Build the constraints
prob += lpSum(players[i] * data.value[data.index[i]] for i in range(len(data))) <= BUDGET # Budget Limit

for pos in POS:
  prob += lpSum(players[i] for i in range(len(data)) if positions[i] == pos) <= pos_available[pos] # Position Limit

for club in CLUBS:
  prob += lpSum(players[i] for i in range(len(data)) if teams[i] == club) <= 3 # Club Limit

In [226]:
prob.solve()

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/mark.obrien/Develop/personal/team-picker/env/lib/python3.9/site-packages/pulp/apis/../solverdir/cbc/osx/64/cbc /var/folders/5y/61922x717ts3bpxvrpkd71bc0000gq/T/59b73fb263d54aafa0c2e872b3295aee-pulp.mps max timeMode elapsed branch printingOptions all solution /var/folders/5y/61922x717ts3bpxvrpkd71bc0000gq/T/59b73fb263d54aafa0c2e872b3295aee-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 30 COLUMNS
At line 3053 RHS
At line 3079 BOUNDS
At line 3638 ENDATA
Problem MODEL has 25 rows, 558 columns and 1511 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 2803 - 0.00 seconds
Cgl0004I processed model has 22 rows, 394 columns (394 integer (393 of which binary)) and 1177 elements
Cutoff increment increased from 1e-05 to 0.9999
Cbc0038I Initial state - 0 integers unsatisfied sum - 7.77156e-16
Cbc

1

In [227]:
# Create a data frame and save picks into it
picks_df = pd.DataFrame()
picks = []
pick_date = datetime.now(timezone.utc).strftime("%Y-%m-%d")

for v in prob.variables():
    if v.varValue != 0:
        picks.append(
            {
                'date_picked': pick_date,
                'player_name': data.name[int(v.name.split("_")[1])],
                'club': data.team[int(v.name.split("_")[1])],
                'position': data.position[int(v.name.split("_")[1])],
                'points': data.total_points[int(v.name.split("_")[1])],
                'price': data.value[int(v.name.split("_")[1])],
                'goals_scored': data.goals_scored[int(v.name.split("_")[1])],
                'assists': data.assists[int(v.name.split("_")[1])],
                'minutes': data.minutes[int(v.name.split("_")[1])],
                'goals_conceded': data.goals_conceded[int(v.name.split("_")[1])],
                'creativity': data.creativity[int(v.name.split("_")[1])],
                'influence': data.influence[int(v.name.split("_")[1])],
                'threat': data.threat[int(v.name.split("_")[1])],
                'bonus': data.bonus[int(v.name.split("_")[1])],
                'bps': data.bps[int(v.name.split("_")[1])],
                'ict_index': data.ict_index[int(v.name.split("_")[1])],
                'clean_sheets': data.clean_sheets[int(v.name.split("_")[1])],
                'red_cards': data.red_cards[int(v.name.split("_")[1])],
                'yellow_cards': data.yellow_cards[int(v.name.split("_")[1])],
                'selected_by_percent': data.selected_by_percent[int(v.name.split("_")[1])]
            }
        )

picks_df = pd.DataFrame(picks)
picks_df.to_csv(f'./data/picks/{pick_date}', index=False)